In [12]:
import sqlite3

connection = sqlite3.connect('example.db')

cursor = connection.cursor()

with open('create_tables.sql', 'r') as file:
    sql_script = file.read()

cursor.executescript(sql_script)

with open('insert_tables.sql', 'r') as file:
    sql_script = file.read()

cursor.executescript(sql_script)


connection.commit()
connection.close()


In [ ]:
connection = sqlite3.connect('example.db')

cursor = connection.cursor()

tables = ["Label", "Artists", "Releases", "Rerelease", "Songs", "AlbumTrack"]

for table in tables:
    print(f"Results from {table}:")
    cursor.execute(f"SELECT * FROM {table}")
    rows = cursor.fetchall()
    for row in rows:
        print(row)
    print("")

# Close the connection
connection.close()


Results from Label:
(1, 'Ultra Records', 'UL')
(2, 'mau5trap', 'MAU')

Results from Artists:
(1, 'deadmau5')

Results from Releases:
(1, 'Random Album Title', 2008, 1)

Results from Rerelease:
('UL 1868-2', 1, '617465186820', 1, 2008, 'CD')
('MAUSRAT', 1, None, 2, 2016, 'Web')

Results from Songs:
(1, 'Brazil (2nd Edit)', 323, None, 1)
(2, 'I Remember', 548, None, 1)
(3, 'Faxing Berlin (Piano Acoustica Version)', 105, 1, 1)
(4, 'Faxing Berlin', 150, None, 1)

Results from AlbumTrack:
(1, 'UL 1868-2', 1, 5)
(1, 'UL 1868-2', 2, 7)
(1, 'UL 1868-2', 3, 8)
(1, 'UL 1868-2', 4, 9)
(1, 'MAUSRAT', 1, 5)
(1, 'MAUSRAT', 2, 7)
(1, 'MAUSRAT', 3, 8)
(1, 'MAUSRAT', 4, 9)



In [17]:
connection = sqlite3.connect('example.db')

cursor = connection.cursor()

queurys = [
"""SELECT s.stitle
FROM Songs s
JOIN AlbumTrack at ON s.sid = at.sid
JOIN Rerelease rr ON at.cat_number = rr.cat_number
JOIN Label l ON rr.label = l.lid
WHERE rr.year > 2008
  AND l.lname = 'Ultra Records';
""",
"""
SELECT r.rtitle, rr.year, COUNT(at.sid) AS album_track_count
FROM Rerelease rr
JOIN AlbumTrack at ON rr.cat_number = at.cat_number
JOIN Releases r ON rr.rid = r.rid
GROUP BY rr.cat_number, rr.year;
""",
"""
SELECT r.rtitle, rr.cat_number, SUM(s.duration) AS total_duration
FROM Rerelease rr
JOIN AlbumTrack at ON rr.cat_number = at.cat_number
JOIN Songs s ON at.sid = s.sid
JOIN Releases r ON rr.rid = r.rid
GROUP BY rr.cat_number, r.rtitle;
""",
"""
SELECT remix.stitle AS remix,
       remix.duration,
       remix_artist.aname AS remixing_artist,
       original.stitle AS original,
       original_artist.aname AS original_artist
FROM Songs remix
JOIN Artists remix_artist ON remix.artist = remix_artist.aid
JOIN Songs original ON remix.remix_of = original.sid
JOIN Artists original_artist ON original.artist = original_artist.aid
WHERE remix.remix_of IS NOT NULL;
"""]
for index, query in enumerate(queurys):
    print(f"Query {index} Results")
    cursor.execute(query)
    rows = cursor.fetchall()
    for row in rows:
        print(row)
    print("")

# Close the connection
connection.close()


Query 0 Results

Query 1 Results
('Random Album Title', 2016, 4)
('Random Album Title', 2008, 4)

Query 2 Results
('Random Album Title', 'MAUSRAT', 1126)
('Random Album Title', 'UL 1868-2', 1126)

Query 3 Results
('Faxing Berlin (Piano Acoustica Version)', 105, 'deadmau5', 'Brazil (2nd Edit)', 'deadmau5')

